In [11]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics, regularizers

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

tf.random.set_seed(0)

vgg13 = [
    layers.Conv2D(64, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    layers.Conv2D(64, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2, 2], strides = 2, padding = "same"),
    
    layers.Conv2D(128, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    layers.Conv2D(128, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2, 2], strides = 2, padding = "same"),
    
    layers.Conv2D(256, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    layers.Conv2D(256, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    # layers.Conv2D(256, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu), 
    layers.MaxPool2D(pool_size = [2, 2], strides = 2, padding = "same"),
    
    layers.Conv2D(512, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    layers.Conv2D(512, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    # layers.Conv2D(512, kernel_size = [3, 3], padding = "same", activation = tf.nn.relu),
    layers.MaxPool2D(pool_size = [2, 2], strides = 2, padding = "same"),
    
]

fc = [
    layers.Dense(256, activation = tf.nn.relu),
    layers.Dense(128, activation = tf.nn.relu),
    layers.Dropout(0.5),
    layers.Dense(10, activation = None),
]

def process(x, y):
    x = tf.cast(x, dtype = tf.float32) / 255.
    y = tf.cast(y, dtype = tf.int32)
    return x, y

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
y_train = tf.squeeze(y_train, axis = 1)
y_test = tf.squeeze(y_test, axis = 1)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(1000).map(process).batch(32)

test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.map(process).batch(32)

def main():
    net = Sequential(vgg13)
    fc_net = Sequential(fc)
    
    net.build(input_shape = [None, 32, 32, 3])
    fc_net.build(input_shape = [None, 2048])
    optimizer = optimizers.Adam(lr = 1e-4)
    
    variables = net.trainable_variables + fc_net.trainable_variables
    for epoch in range(30):
        for step, (x, y) in enumerate(train_data):
            with tf.GradientTape() as tape:
                out = net(x)
                out = tf.reshape(out, [-1, 2048])
                logits = fc_net(out)
                Y = tf.one_hot(y, depth = 10)
                loss = tf.losses.categorical_crossentropy(Y, logits, from_logits = True)
                loss = tf.reduce_mean(loss)
            grads = tape.gradient(loss, variables)
            optimizer.apply_gradients(zip(grads, variables))
            
            if step % 100 == 0:
                print(epoch, step, 'loss', float(loss))
    
        total = 0
        ac = 0
        for (x, y) in test_data:
            out = net(x)
            out = tf.reshape(out, [-1, 2048])
            logits = fc_net(out)
            prob = tf.nn.softmax(logits, axis = 1)
            pred = tf.argmax(prob, axis = 1)
            pred = tf.cast(pred, dtype = tf.int32)
            correct = tf.cast(tf.equal(pred, y), dtype = tf.int32)
            correct = tf.reduce_sum(correct)
            total += x.shape[0]
            ac += int(correct)
        acc = ac / total
        print(epoch, 'accruacy: ', acc)
        
if __name__ == '__main__':
    main()

0 0 loss 2.3030176162719727
0 100 loss 2.219264507293701
0 200 loss 1.9556794166564941
0 300 loss 1.956813097000122
0 400 loss 1.8391170501708984
0 500 loss 1.7429561614990234
0 600 loss 1.6545085906982422
0 700 loss 1.5768307447433472
0 800 loss 1.4659579992294312
0 900 loss 1.3519532680511475
0 1000 loss 1.6038812398910522
0 1100 loss 1.78470778465271
0 1200 loss 1.2915844917297363
0 1300 loss 1.5890097618103027
0 1400 loss 1.097398281097412
0 1500 loss 1.326195240020752
0 accruacy:  0.5489
1 0 loss 1.2197494506835938
1 100 loss 1.2578954696655273
1 200 loss 1.1089178323745728
1 300 loss 1.5312414169311523
1 400 loss 1.2294416427612305
1 500 loss 1.0616861581802368
1 600 loss 1.0130943059921265
1 700 loss 0.9965970516204834
1 800 loss 1.511429786682129
1 900 loss 1.2414414882659912
1 1000 loss 0.9643637537956238
1 1100 loss 1.0630580186843872
1 1200 loss 1.0899286270141602
1 1300 loss 1.1711299419403076
1 1400 loss 0.7736486196517944
1 1500 loss 1.0905522108078003
1 accruacy:  0.6214

15 1400 loss 0.012278186157345772
15 1500 loss 0.00714449817314744
15 accruacy:  0.7964
16 0 loss 0.015818465501070023
16 100 loss 0.011301486752927303
16 200 loss 0.20196859538555145
16 300 loss 0.016271773725748062
16 400 loss 0.10128629207611084
16 500 loss 0.0038722832687199116
16 600 loss 0.05524285510182381
16 700 loss 0.031967051327228546
16 800 loss 0.04056238383054733
16 900 loss 0.03464455530047417
16 1000 loss 0.11395113170146942
16 1100 loss 0.006457194685935974
16 1200 loss 0.07797442376613617
16 1300 loss 0.058632005006074905
16 1400 loss 0.006874356884509325
16 1500 loss 0.025725437328219414
16 accruacy:  0.7955
17 0 loss 0.06164376810193062
17 100 loss 0.16633255779743195
17 200 loss 0.008514674380421638
17 300 loss 0.010654594749212265
17 400 loss 0.0017497382359579206
17 500 loss 0.036298640072345734
17 600 loss 0.11061179637908936
17 700 loss 0.01228165440261364
17 800 loss 0.0822848528623581
17 900 loss 0.09501102566719055
17 1000 loss 0.01109826285392046
17 1100 lo